<a href="https://colab.research.google.com/github/paranjaa/ece1508-student-depression-project/blob/main/ECE1508_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
print("Hello World")

Hello World


In [70]:
#import each one of the libraries
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [71]:
#have to go through permissions each time this starts up
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [72]:
#need to check if this works with others
df = pd.read_csv("gdrive/My Drive/ECE1508/Project/Student_Depression_Dataset.csv")


In [73]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  object 
 11  Di

In [74]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [75]:
# id: not sure why its numbered like that, probably not useful
#dropping it for now, might need it later
df = df.drop('id', axis=1)

In [76]:

## Gender: convert to one hot columns? maybe useful. Gender ratio is at 0.44, not so bad
# df["Gender"].value_counts()
#putting it in lowercase first
df = df.rename(columns={'Gender': 'gender'})
df["gender"] = df["gender"].map({"Female": True, "Male": False})

#also putting in lowercase

# df.head()

In [77]:
## Age: convert from float to int, no decimals. Mostly in 20s, maybe find an average
# df["Age"].value_counts()
df = df.rename(columns={'Age': 'age'})

#we don't need numbers that are too large, keeping it as int8
df["age"] = df["age"].astype('int8')


In [78]:
# City: convert to one hot columns. Put the couple dozen cities with one or two students in "other"
# df["City"].value_counts()

city_count = df["City"].value_counts()

#getting cities that are in less than 10 data points
threshold = 10
rare_cities = city_count[city_count < threshold].index

#replace the values in City where there's something in the rare cities list with other
#got this partially from gpt, verify how it works
df["City"] = df["City"].where(~df["City"].isin(rare_cities), "Other")
city_count = df["City"].value_counts()
# df["City"].value_counts()

#get a data frame of one-hot columns of the cities
cities_df = pd.get_dummies(df["City"], prefix="city")
# cities_df.head()

#then add it onto the side of the original dataframe
df = pd.concat([df, cities_df], axis=1)
#remove the original cities column
df = df.drop('City', axis=1)

In [79]:
## Profession: not useful, barely anyone put anything other than "student"
# df["Profession"].value_counts()

#also dropping this one?
df = df.drop('Profession', axis=1)

In [80]:
## Academic Pressure: convert to int. 3 is most popular, followed by 5, then 4.
# df["Academic Pressure"].value_counts()
df = df.rename(columns={'Academic Pressure': 'academic_pressure'})
df["academic_pressure"] = df["academic_pressure"].astype(int)

In [81]:
# Work Pressure: Not useful. Only 3 people put anything other than 0
# df["Work Pressure"].value_counts()
df = df.drop('Work Pressure', axis=1)

In [82]:
## CGPA: Need to keep this as a float? There's 322 distinct values, but it seems useful.
## maybe separate it into brackets though
# df["CGPA"].value_counts()

#not touching this one for now, just putting it in lowercase
df = df.rename(columns={'CGPA': 'cgpa'})


In [83]:
# Study Satisfaction: Convert to int. Suprisingly close values, save for people who put 0.0
# df["Study Satisfaction"].value_counts()

df = df.rename(columns={'Study Satisfaction': 'study_satisfaction'})
df["study_satisfaction"] = df["study_satisfaction"].astype(int)


In [84]:
# Job Satisfaction: Not useful, assuming that 0.0 means N.A, and so many people put student
# df["Job Satisfaction"].value_counts()

df = df.drop('Job Satisfaction', axis=1)

In [85]:
## Sleep Duration: Convert to numeric values and one hot columns?
## it's in a weird format with things in buckets
# df["Sleep Duration"].value_counts()

#making one hot columns for these
sleep_df = pd.get_dummies(df['Sleep Duration'], prefix='sleep')
#adding them back onto the side, removing original
df = pd.concat([df, sleep_df], axis=1)
#figure out some way to name them better later on
df = df.drop('Sleep Duration', axis=1)

In [86]:
# Dietary Habits: Need one hot columns. "Unhealthy" is most popular, but only by <500
# df["Dietary Habits"].value_counts()
diet_df = pd.get_dummies(df['Dietary Habits'], prefix='diet')
df = pd.concat([df, diet_df], axis=1)
df = df.drop('Dietary Habits', axis=1)


In [87]:
## Degree: Lots of categories. Put into one hot columns? It'll make a lot of them
# df["Degree"].value_counts()
degree_df = pd.get_dummies(df['Degree'], prefix='degree')
df = pd.concat([df, degree_df], axis=1)
df = df.drop('Degree', axis=1)

In [88]:
## Have you ever had suicidal thoughts ? : Convert to binary. Almost 60% say yes, which I guess could be worse
## Also rename to something more concise than all that

# df["Have you ever had suicidal thoughts ?"].value_counts()
df = df.rename(columns={'Have you ever had suicidal thoughts ?': 'suicidal_thoughts'})
df["suicidal_thoughts"] = df["suicidal_thoughts"].map({"Yes": True, "No": False})


In [89]:
## Work/Study Hours: Most people are in the range of 10-12 hours. Convert to int.
# df["Work/Study Hours"].value_counts()
df = df.rename(columns={'Work/Study Hours': 'work_study_hours'})
df["work_study_hours"] = df["work_study_hours"].astype('int8')

In [90]:
# Financial Stress: Suprisngly close, with 5.0 on top. Convert to int.
df = df.rename(columns={'Financial Stress': 'financial_stress'})
df["financial_stress"].value_counts()
#wait, this column has two values that aren't filled in, it has 27898 entries?
#filling the handful of NA values with a 5, since it's already ahead by 900 or so
df["financial_stress"] = df["financial_stress"].fillna(5.0)

df["financial_stress"] = df["financial_stress"].astype(int)

In [91]:
# Family History of Mental Illness: Also quite close numbers, convert to binary
#also rename this one
# df["Family History of Mental Illness"].value_counts()

df = df.rename(columns={"Family History of Mental Illness": "family_history"})
df["family_history"] = df["family_history"].map({"Yes": True, "No": False})

In [92]:
# Depression: Main label. More people do than don't.
# df["Depression"].value_counts()

df = df.rename(columns={"Depression": "depression"})
df["depression"] = df["depression"].astype(int)

#put depression at the end of the list so we can get at it easily
final_column = df.pop('depression')
df['depression'] = final_column


In [93]:
#final name cleanup, put everything else in lowercase, as well as replacing any spaces
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("-", "_")

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 78 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gender                   27901 non-null  bool   
 1   age                      27901 non-null  int8   
 2   academic_pressure        27901 non-null  int64  
 3   cgpa                     27901 non-null  float64
 4   study_satisfaction       27901 non-null  int64  
 5   suicidal_thoughts        27901 non-null  bool   
 6   work_study_hours         27901 non-null  int8   
 7   financial_stress         27901 non-null  int64  
 8   family_history           27901 non-null  bool   
 9   city_agra                27901 non-null  bool   
 10  city_ahmedabad           27901 non-null  bool   
 11  city_bangalore           27901 non-null  bool   
 12  city_bhopal              27901 non-null  bool   
 13  city_chennai             27901 non-null  bool   
 14  city_delhi            

In [27]:
df.head()

,gender,age,academic_pressure,cgpa,study_satisfaction,suicidal_thoughts,work_study_hours,financial_stress,family_history,city_agra,...,degree_mba,degree_mbbs,degree_mca,degree_md,degree_me,degree_mhm,degree_msc,degree_others,degree_phd,depression
0,False,33,5,8.97,2,True,3,1,False,False,...,False,False,False,False,False,False,False,False,False,1
1,True,24,2,5.90,5,False,3,2,True,False,...,False,False,False,False,False,False,False,False,False,0
2,False,31,3,7.03,5,False,9,1,True,False,...,False,False,False,False,False,False,False,False,False,0
3,True,28,3,5.59,2,True,4,5,True,False,...,False,False,False,False,False,False,False,False,False,1
4,True,25,4,8.13,3,True,1,1,False,False,...,False,False,False,False,False,False,False,False,False,0


In [95]:
X = df.iloc[:, 0:-1].values #get everything except for the last column, done the cleanup
y = df.iloc[:, -1].values   # only take the last one, since it has our label

In [96]:
#might want to find a better way to do this, its weird that train_test_split is its own thing,
#like tensorflow just has .sample()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#some weird conversion with going to numpy arrays?
X_train = torch.tensor(X_train.astype(np.float32))
X_test = torch.tensor(X_test.astype(np.float32))

#similar kind of reshaping for binary classification
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [97]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

#need to look into how dataloaders work, also that the test set doesnt need to be shuffled
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

This won't work for now, done a bunch of stuff to the datset, so it doesn't fit at the moment

In [98]:
#@title
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        #probably want to mess with the sizes here to better fit the model?
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [99]:

model = SimpleNN(X_train.shape[1])
criterion = nn.BCELoss()  #also tweek this learning
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [100]:
num_epochs = 20
train_losses = []

for epoch in range(num_epochs):
    epoch_loss = 0  # Track loss for this epoch
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()  # Sum up batch losses

    avg_loss = epoch_loss / len(train_loader)  # Average loss per epoch
    train_losses.append(avg_loss)  # Store for graphing

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/20, Loss: 0.3801
Epoch 2/20, Loss: 0.3618
Epoch 3/20, Loss: 0.3580
Epoch 4/20, Loss: 0.3561
Epoch 5/20, Loss: 0.3553
Epoch 6/20, Loss: 0.3543
Epoch 7/20, Loss: 0.3518
Epoch 8/20, Loss: 0.3511
Epoch 9/20, Loss: 0.3507
Epoch 10/20, Loss: 0.3511
Epoch 11/20, Loss: 0.3500
Epoch 12/20, Loss: 0.3505
Epoch 13/20, Loss: 0.3491
Epoch 14/20, Loss: 0.3487
Epoch 15/20, Loss: 0.3491
Epoch 16/20, Loss: 0.3490
Epoch 17/20, Loss: 0.3488
Epoch 18/20, Loss: 0.3470
Epoch 19/20, Loss: 0.3475
Epoch 20/20, Loss: 0.3486


In [101]:
#probably would have been useful to know doing it like this during a2
model.eval()
with torch.no_grad():
    correct, total = 0, 0
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = (outputs >= 0.5).float()
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8352


In [103]:
plt.plot(range(1, num_epochs + 1), train_losses, marker=o, linestyle=-, color=b)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Time")
plt.show()

#well, this needs improvements, its not converging that well at all

SyntaxError: invalid syntax (<ipython-input-103-ff9a3b4e25b8>, line 1)

In [ ]:
#old code based on a previous project I did. Doesnt work, so I went and used an LLM to get some example code and made the section above
#definitely want to do something better though

# @title
# print("Hello World")
# from google.colab import drive
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# csv_file = drive/My Drive/Colab_Notebooks/ECE 1508/Project/Datasets/Student Depression Dataset.csv

# student_data = pd.read_csv(csv_file)
# student_df = pd.read_csv(csv_file)


# student_data.head()
# # student_df.info()




# #make a training set
# student_train = student_data.sample(frac=0.7,random_state=100)
# #also a test set
# student_test = student_train.drop(student_train.index)


# #get our label from the dataset (just the depression column)
# #convert it to boolean
# depression_train = np.array(student_train[Depression])
# depression_train = depression_train.astype(bool)


# #probably need to change some values here?
# #like, converting yes to 1,
# input_data = np.array(student_train[[
#    Study Satisfaction,
#     ]])

# input_data = input_data.astype(float)



# x_train = input_data.reshape(-1,1)
# y_train = depression_train




# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(units=1, activation=None),
#   tf.keras.layers.Dense(units=3, activation=None),
#   tf.keras.layers.Dense(units=5, activation=None),

# ])

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
# loss=tf.keras.losses.mae)


# #not sure Im setting this up right, maybe swtich back to pytorch?
# model.fit(x_train,y_train,epochs=1)



# model.summary()


# history = model.fit(x_train,y_train,epochs=100)



#Trained Neural Network 1#

In [ ]:
##